<a href="https://colab.research.google.com/github/BChun11/DATA3001/blob/main/DATA3001_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import hashlib
from scipy.stats import randint

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
# Download all the files into google colab environment
!git clone https://github.com/nokuik/KDDI-IoT-2019.git

Cloning into 'KDDI-IoT-2019'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 42 (delta 1), reused 9 (delta 1), pack-reused 33
Receiving objects: 100% (42/42), 776.84 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (31/31), done.


In [3]:
# List all '.tar.gz files in ipfix directory
%cd KDDI-IoT-2019
%cd ipfix
!ls *.tar.gz

/content/KDDI-IoT-2019
/content/KDDI-IoT-2019/ipfix
 amazon_echo_gen2.tar.gz		        nature_remo.tar.gz
 au_network_camera.tar.gz		        panasonic_doorphone.tar.gz
 au_wireless_adapter.tar.gz		        philips_hue_bridge.tar.gz
 bitfinder_awair_breathe_easy.tar.gz	       'planex_camera_one_shot!.tar.gz'
 candy_house_sesami_wi-fi_access_point.tar.gz   planex_smacam_outdoor.tar.gz
 irobot_roomba.tar.gz			        planex_smacam_pantilt.tar.gz
 jvc_kenwood_cu-hb1.tar.gz		        powerelectric_wi-fi_plug.tar.gz
 jvc_kenwood_hdtv_ip_camera.tar.gz	        qrio_hub.tar.gz
 line_clova_wave.tar.gz			        sony_network_camera.tar.gz
 link_japan_eremote.tar.gz		        sony_smart_speaker.tar.gz
 mouse_computer_room_hub.tar.gz		        xiaomi_mijia_led.tar.gz


In [4]:
# List all tar.gz files and store them in a variable
files = !ls -1 *.tar.gz

# Extract each tar.gz file
for file in files:
    print(f"Extract {file} ")
    !tar -xzvf {file}

Extract amazon_echo_gen2.tar.gz 
amazon_echo_gen2.json
Extract au_network_camera.tar.gz 
au_network_camera.json
Extract au_wireless_adapter.tar.gz 
au_wireless_adapter.json
Extract bitfinder_awair_breathe_easy.tar.gz 
bitfinder_awair_breathe_easy.json
Extract candy_house_sesami_wi-fi_access_point.tar.gz 
candy_house_sesami_wi-fi_access_point.json
Extract irobot_roomba.tar.gz 
irobot_roomba.json
Extract jvc_kenwood_cu-hb1.tar.gz 
jvc_kenwood_cu-hb1.json
Extract jvc_kenwood_hdtv_ip_camera.tar.gz 
jvc_kenwood_hdtv_ip_camera.json
Extract line_clova_wave.tar.gz 
line_clova_wave.json
Extract link_japan_eremote.tar.gz 
link_japan_eremote.json
Extract mouse_computer_room_hub.tar.gz 
mouse_computer_room_hub.json
Extract nature_remo.tar.gz 
nature_remo.json
Extract panasonic_doorphone.tar.gz 
panasonic_doorphone.json
Extract philips_hue_bridge.tar.gz 
philips_hue_bridge.json
Extract 'planex_camera_one_shot!.tar.gz' 
planex_camera_one_shot!.json
Extract planex_smacam_outdoor.tar.gz 
planex_smacam

In [5]:
# Print the current working directory
print("Current Working Directory:", os.getcwd())

# List the contents of the current working directory
print("Contents of Current Directory:", os.listdir())

!cd

Current Working Directory: /content/KDDI-IoT-2019/ipfix
Contents of Current Directory: ['philips_hue_bridge.tar.gz', 'sony_network_camera.tar.gz', 'jvc_kenwood_hdtv_ip_camera.json', 'qrio_hub.tar.gz', 'au_wireless_adapter.json', 'powerelectric_wi-fi_plug.tar.gz', 'jvc_kenwood_hdtv_ip_camera.tar.gz', 'sony_smart_speaker.tar.gz', 'qrio_hub.json', 'panasonic_doorphone.tar.gz', 'jvc_kenwood_cu-hb1.tar.gz', 'jvc_kenwood_cu-hb1.json', 'panasonic_doorphone.json', 'planex_smacam_pantilt.tar.gz', 'i-o_data_qwatch.tar.gz00', 'planex_camera_one_shot!.json', 'au_network_camera.json', 'nature_remo.tar.gz', 'amazon_echo_gen2.json', 'bitfinder_awair_breathe_easy.json', 'planex_smacam_outdoor.tar.gz', 'powerelectric_wi-fi_plug.json', 'sony_network_camera.json', 'planex_smacam_outdoor.json', 'amazon_echo_gen2.tar.gz', 'mouse_computer_room_hub.tar.gz', 'candy_house_sesami_wi-fi_access_point.json', 'mouse_computer_room_hub.json', 'planex_smacam_pantilt.json', 'bitfinder_awair_breathe_easy.tar.gz', 'sony_

In [6]:
# Code to generate distinct tables for each json file using a limited subset

# Define the directory where the JSON files are located
json_directory = '/content/KDDI-IoT-2019/ipfix'

# Get the list of all JSON files in the directory
json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]

# Create distinct tables for each json file
tables = {}
for json_file in json_files:
    # strip .json suffix from device names
    device_name = json_file.split('.')[0]
    # Construct the full path to the JSON file
    json_path = os.path.join(json_directory, json_file)
    # Read the JSON file into a DataFrame, normalize the 'flows' column, and get the first 1000 rows
    df = pd.json_normalize(pd.read_json(json_path, lines=True, nrows=1000)['flows'])
    # Label the DataFrame with the device name
    df['Device'] = device_name
    tables[device_name] = df

# Concatenate all the Dataframes in the tables dictionary into a single Dataframe
df = pd.concat(tables.values(), ignore_index=True)
df.head()


,flowStartMilliseconds,flowEndMilliseconds,flowDurationMilliseconds,reverseFlowDeltaMilliseconds,protocolIdentifier,sourceIPv4Address,sourceTransportPort,packetTotalCount,octetTotalCount,flowAttributes,...,reverseNonEmptyPacketCount,reverseDataByteCount,reverseAverageInterarrivalTime,reverseFirstNonEmptyPacketSize,reverseLargePacketCount,reverseMaxPacketSize,reverseStandardDeviationPayloadLength,reverseStandardDeviationInterarrivalTime,reverseBytesPerPacket,Device
0,2019-06-25 08:36:35.447,2019-06-25 08:36:36.027,0.580,0.0,17,192.168.1.1,33611,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jvc_kenwood_hdtv_ip_camera
1,2019-06-25 08:37:38.253,2019-06-25 08:37:38.833,0.580,0.0,17,192.168.1.1,53885,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jvc_kenwood_hdtv_ip_camera
2,2019-06-25 08:38:40.977,2019-06-25 08:38:41.556,0.579,0.0,17,192.168.1.1,56915,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jvc_kenwood_hdtv_ip_camera
3,2019-06-25 08:39:43.560,2019-06-25 08:39:44.137,0.577,0.0,17,192.168.1.1,52583,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jvc_kenwood_hdtv_ip_camera
4,2019-06-25 08:40:46.447,2019-06-25 08:40:47.026,0.579,0.0,17,192.168.1.1,56244,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jvc_kenwood_hdtv_ip_camera


In [7]:
"""# Reference: https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

 Reduce Memory Usage
  This code is used to reduce memory usage of our dataframe useful when running
  large datasets by reducing the size of the properties of the dataset by selecting
  smaller datatypes


def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df"""

'# Reference: https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN\n\n Reduce Memory Usage\n  This code is used to reduce memory usage of our dataframe useful when running\n  large datasets by reducing the size of the properties of the dataset by selecting\n  smaller datatypes\n\n\ndef reduce_memory_usage(df):\n    start_mem = df.memory_usage().sum() / 1024**2\n    print(\'Memory usage of dataframe is {:.2f} MB\'.format(start_mem))\n\n    for col in df.columns:\n        col_type = df[col].dtype.name\n        if ((col_type != \'datetime64[ns]\') & (col_type != \'category\')):\n            if (col_type != \'object\'):\n                c_min = df[col].min()\n                c_max = df[col].max()\n\n                if str(col_type)[:3] == \'int\':\n                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:\n                        df[col] = df[col].astype(np.int8)\n                    elif c_min > np.iinfo(np.int16).min and c_max

In [8]:
# Apply the reduce memory usage function to our dataset
#df = reduce_memory_usage(df)
#df.info()

### Data Transformation

In [9]:
# Define the list of columns to be dropped
drop_columns = ['flowStartMilliseconds',
                'flowEndMilliseconds',
                'sourceMacAddress',
                'destinationMacAddress'
]

# Drop the columns from the dataset
df = df.drop(columns=drop_columns)

In [10]:
"""
  Encode and transform categorical values since Decision Tree & Random Forest can't handle non-numeric
  categorical data directly
"""
# Ordinal encoding for IP addresses
def ip_to_ordinal(df, col_name):
    # Generate unique codes for each unique IP address
    codes, uniques = pd.factorize(df[col_name])
    df[col_name] = codes
    return df

df = ip_to_ordinal(df, 'sourceIPv4Address')
df = ip_to_ordinal(df, 'destinationIPv4Address')

# Convert hex to int
for col in ['tcpSequenceNumber', 'reverseTcpSequenceNumber', 'vlanId', 'ipClassOfService']:
    df[col] = df[col].apply(lambda x: int(x, 16))

# Label Encoding for the other categorical attributes
label_encoders = {}
for col in ['flowAttributes', 'initialTCPFlags', 'unionTCPFlags', 'reverseInitialTCPFlags', 'reverseUnionTCPFlags', 'reverseFlowAttributes', 'collectorName', 'flowEndReason', 'firstEightNonEmptyPacketDirections', 'Device']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [11]:
# Separate the Device column as label
labels_df = df['Device'].copy()

# Drop the Device column from the original DataFrame
df = df.drop(columns=['Device'])

X = df
y = labels_df

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Applying the Decision Tree and Random Forest Algorithm

#### Assessing the Performance of the Multi-Class Classifier
- The print_score function is designmed to evaluate a multi-class classifier's performance using F1 score and ROC-AUC curve.
- We're using the 'weighted' average for the F1 score and 'ovr' (One-vs-rest) approach for the multi-class ROC-AUC.

In [12]:
# Evaluate how well a classifier model is performing on the training data
def print_score(cls, X_train, y_train, X_test, y_test, train=True):
    if train:
        X, y = X_train, y_train
        data_type = "Train"
    else:
        X, y = X_test, y_test
        data_type = "Test"

    # The model 'cls' uses the features X to make predictions
    pred_y = cls.predict(X)
    prob = cls.predict_proba(X)

    # Compute 'weighted' F1-Score
    f1Score = f1_score(y, pred_y, average='weighted')
    # Compute ROC-AUC for each class using 'One-vs-rest'
    roc_auc = roc_auc_score(y, prob, average='weighted', multi_class='ovr')

    # Print results
    print(f"=== {data_type} Data ===")
    print(f"Weighted F1 Score = {f1Score:.4f}\n")
    print(f'ROC-AUC score: {roc_auc:.4f}\n')
    print("_______________________________________________")
    print(classification_report(y, pred_y))
    print("\n")

Treating missing values: Using Imputation to fill in the missing values

In [13]:
# Using imputation to fill in the missing values
# Create impute object
imputer = SimpleImputer()

# Fit imputer on training data
imputer.fit(X_train)

# Impute training data
X_train = imputer.transform(X_train)

# Impute the test data
X_test = imputer.transform(X_test)

### Decision Tree Classifier

In [14]:
# Create Decision Tree Classifier object
dec_tree = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
dec_tree.fit(X_train, y_train)

# Print model performance on training data
print_score(dec_tree, X_train, y_train, X_test, y_test, train=True)

# Print model performance on test data
print_score(dec_tree, X_train, y_train, X_test, y_test, train=False)

=== Train Data ===
Weighted F1 Score = 1.0000

ROC-AUC score: 1.0000

_______________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       897
           1       1.00      1.00      1.00       899
           2       1.00      1.00      1.00       914
           3       1.00      1.00      1.00       889
           4       1.00      1.00      1.00       910
           5       1.00      1.00      1.00       916
           6       1.00      1.00      1.00       879
           7       1.00      1.00      1.00       905
           8       1.00      1.00      1.00       915
           9       1.00      1.00      1.00       897
          10       1.00      1.00      1.00       896
          11       1.00      1.00      1.00       895
          12       1.00      1.00      1.00       899
          13       1.00      1.00      1.00       906
          14       1.00      1.00      1.00       894
          15     

### Random Forest

In [25]:
# Create Random Forest Classifier object
rf_cls = RandomForestClassifier(random_state=42)

# Fit the model to the training data
rf_cls.fit(X_train, y_train)

# Print model performance on training data
print_score(rf_cls, X_train, y_train, X_test, y_test, train=True)

# Print model performance on test data
print_score(rf_cls, X_train, y_train, X_test, y_test, train=False)

=== Train Data ===
Weighted F1 Score = 1.0000

ROC-AUC score: 1.0000

_______________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       897
           1       1.00      1.00      1.00       899
           2       1.00      1.00      1.00       914
           3       1.00      1.00      1.00       889
           4       1.00      1.00      1.00       910
           5       1.00      1.00      1.00       916
           6       1.00      1.00      1.00       879
           7       1.00      1.00      1.00       905
           8       1.00      1.00      1.00       915
           9       1.00      1.00      1.00       897
          10       1.00      1.00      1.00       896
          11       1.00      1.00      1.00       895
          12       1.00      1.00      1.00       899
          13       1.00      1.00      1.00       906
          14       1.00      1.00      1.00       894
          15     

### Random Forest Hyperparameter tuning

In [30]:
# Define the hyperparameters and their possible values
param_dist = {
    'n_estimators': randint(50, 250),  # Number of trees in the forest
    'max_depth': [None] + list(randint(5, 50).rvs(10)),  # Maximum depth of the tree
    'min_samples_split': randint(2, 50),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 50),  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False],  # Method of selecting samples for training each tree
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'class_weight': [None, 'balanced']  # Handle potential imbalance in classes
}

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=10, scoring='f1_weighted',
                                   cv=5, verbose=2, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters from the Random Search
best_random = random_search.best_estimator_
print(f"Best Parameters: {random_search.best_params_}")

# Evaluate the best model
print_score(best_random, X_train, y_train, X_test, y_test, train=True)
print_score(best_random, X_train, y_train, X_test, y_test, train=False)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, class_weight=balanced, max_depth=14, max_features=auto, min_samples_leaf=21, min_samples_split=40, n_estimators=171; total time=   2.4s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, class_weight=balanced, max_depth=14, max_features=auto, min_samples_leaf=21, min_samples_split=40, n_estimators=171; total time=   2.5s


...

[CV] END bootstrap=False, class_weight=balanced, max_depth=29, max_features=auto, min_samples_leaf=8, min_samples_split=48, n_estimators=84; total time=   1.8s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=False, class_weight=balanced, max_depth=29, max_features=auto, min_samples_leaf=8, min_samples_split=48, n_estimators=84; total time=   1.9s
[CV] END bootstrap=False, class_weight=None, max_depth=27, max_features=sqrt, min_samples_leaf=40, min_samples_split=5, n_estimators=51; total time=   1.0s
[CV] END bootstrap=False, class_weight=None, max_depth=27, max_features=sqrt, min_samples_leaf=40, min_samples_split=5, n_estimators=51; total time=   1.0s
[CV] END bootstrap=False, class_weight=None, max_depth=27, max_features=sqrt, min_samples_leaf=40, min_samples_split=5, n_estimators=51; total time=   0.9s
[CV] END bootstrap=False, class_weight=None, max_depth=27, max_features=sqrt, min_samples_leaf=40, min_samples_split=5, n_estimators=51; total time=   0.9s
[CV] END bootstrap=False, class_weight=None, max_depth=27, max_features=sqrt, min_samples_leaf=40, min_samples_split=5, n_estimators=51; total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'bootstrap': False, 'class_weight': None, 'max_depth': 40, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 25, 'n_estimators': 207}
=== Train Data ===
Weighted F1 Score = 0.9891

ROC-AUC score: 0.9999

_______________________________________________
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       897
           1       0.91      0.99      0.95       899
           2       1.00      0.96      0.98       914
           3       0.99      0.99      0.99       889
           4       0.99      1.00      1.00       910
           5       1.00      1.00      1.00       916
           6       1.00      0.94      0.97       879
           7       1.00      1.00      1.00       905
           8       1.00      1.00      1.00       915
           9       1.00      1.00      1.00       897
          10       1.00      1.00      1.00       896
          11       0.99      1.00      0.99       895
         